In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
import time
from urllib.request import Request, urlopen

In [2]:
url = 'https://www.imdb.com/search/title/?genres=thriller&sort=boxoffice_gross_us,desc&explore=title_type,genres'
response = requests.get(url)
time.sleep(4)

In [3]:
response.status_code

200

In [4]:
# save HTML from requests 
page_html = response.text

In [5]:
soup = bs(page_html)

In [6]:
soup

<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Thriller
(Sorted by US Box Office Descending) - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/search/title/?genres=thriller" rel="canonical"/>
<meta content="http://www.i

#### Overall Strategy:
Identify the parent div that contains the information we want. We're casting the net wide here so that it contains children of data that we may need so we can use one loop. 

We put this div in a variable with the structure shown below and loop through it. 

The variables within the loop are the coordinates to the specific information we want to capture in the larger div. 

#### Note 1
For content that is under a few levels from the parent level, consider using going up a level first then drill down, which in your case is the ratings-bar div, then from that look at the text from within the inline-block ratings-imdb-rating div.

Mike called out that you the p tag can contain a lot of information and we can use that locator as a way to capture a lot of information we may need. 

In [7]:
# code that references note 1
# user_ratings = content.find('div', class_ = 'ratings-bar')
# user_rating = user_ratings.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')

In [16]:
d = {}

specs = soup.find_all('div',{'class':'lister-item-content'})

                                   
for content in specs:
    
    movie_year = content.find('span',class_='lister-item-year text-muted unbold').text.replace("(", "").replace(")", "") 
        
    movie_title = content.find_all('a')[0].text
    
    
    user_ratings = content.find('div', class_ = 'ratings-bar')
    user_rating = user_ratings.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
    
    metascore = content.find('div', class_ = "inline-block ratings-metascore").text.replace('\n', '')
    
    # movie deets contains many features. Below we get them indiviudally 
    movie_deets = content.find_all('p')[0]
    movie_runtime = movie_deets.find('span', class_='runtime').text
    movie_genre = movie_deets.find('span', class_='genre').text.replace('\n','').strip()
    movie_rating = movie_deets.find('span', class_ = 'certificate').text
    
    
    # votes and gross box office $
    
    # first attempt:
    #votes_gross = content.find('p', class_= "sort-num_votes-visible").text.strip().split('|')
    votes_gross = content.find('p', class_= "sort-num_votes-visible").text
    votes_gross_split = votes_gross.split('$')[1]
    final_gross = float(votes_gross_split[:-2])
    
    
    # another attempt
#     votes_gross = content.find('p', class_= "sort-num_votes-visible").find_all(attrs={"name" : "nv"})
#     votes_gross = [el['data-value'] for el in us_gross]
    
    # Director / Stars Work 
    #gets tags 
    director_stars = content.find_all('p')[2]
    
    # This is getting me the director separated from stars not sure how! 
    director = director_stars.find('a').text
 
    d['Title'] = movie_title
    d['Director'] = director
    d['Run Time'] = movie_runtime 
    d['Movie Rating'] = movie_rating
    d['Genre'] = movie_genre
    d['US Gross'] = final_gross
    

# not needed at this time:
#     d['Year'] = movie_year
#     d['User Rating'] = user_rating
#     d['Metascore'] =  metascore
    
    #print(final_gross)
    #print(director_stars)
    #print(director)
    #print(type(votes_gross))
    #print(us_gross)
    #print(us_gross_lst)
    #print(movie_deets)
    #print(gross)
    #print(votes)
    
d

{'Title': 'Mission: Impossible',
 'Director': 'Brian De Palma',
 'Run Time': '110 min',
 'Movie Rating': 'PG-13',
 'Genre': 'Action, Adventure, Thriller',
 'US Gross': 180.98}

In [14]:
d

{'Title': 'Mission: Impossible',
 'Director': 'Brian De Palma',
 'Run Time': '110 min',
 'Movie Rating': 'PG-13',
 'Genre': 'Action, Adventure, Thriller',
 'US Gross': 180.98}

In [15]:
batch1_movies = pd.DataFrame(d, index=[1])
batch1_movies

,Title,Director,Run Time,Movie Rating,Genre,US Gross
1,Mission: Impossible,Brian De Palma,110 min,PG-13,"Action, Adventure, Thriller",180.98


### Notes from Mike

In [120]:
# for content in specs:
#     movie_detail = {}
#     movie_year = content.find('span',class_='lister-item-year text-muted unbold').text \
#                             .replace('(','') \
#                             .replace(')','') 
#     movie_title = content.find_all('a')[0].text
#     movie_ratings = content.find('div', class_ = 'ratings-bar')
#     movie_rating = movie_ratings.find('div', class_ = 'inline-block ratings-imdb-rating').text \
#                             .replace('\n','')
#     movie_deets = content.find_all('p')[0]
#     movie_runtime = movie_deets.find('span', class_='runtime').text
#     movie_genre = movie_deets.find('span', class_='genre').text.replace('\n','').strip()
 
#     #print(movie_year)
#     #print(movie_title)
#     #print(movie_rating)
#     #print(movie_deets)
#     #print('---')
#     #print(movie_desc)
#     movie_detail['Title'] = movie_title
#     movie_detail['Year'] = movie_year
#     movie_detail['User_Rating'] = movie_rating
#     movie_detail['Runtime'] = movie_runtime
#     movie_detail['Genre'] = movie_genre

#     movie_details.append(movie_detail)

Status List of Features Captured
- y Movie Title
- y US Box Office Gross
- y Run Time
- y Rating
- n Stars
- y Director 

- y Genre Type of Thriller
- y Metascore - needs clean up
- y User Rating 
- n Number of Votes
- y Year